This is a **non-offical** material for Lab3 just for your reference

I am trying to write some Python codes that are functionally similiar to C you will use in Labs. However, I cannot guarantee that.

In [1]:
import threading
import time

# 1. ```ex_mutex``` in Python
## 1.1. Body function
I don't like global variables, so I use an ojbect to manage my thread bodies and payloads

* ```thread_body()``` maps to the ```body()``` function
* ```payloads``` maps to the static variables in C example
* ```threading.Lock()``` maps to the ```mutex``` in pthreads

In [2]:
class MutexExample():
    def __init__(self, payload0, payload1, locks):
        self.payloads = [payload0, payload1]
        self.locks = locks
        
    def thread_body(self, payload_id, lock_id, opcode, identifier, lock_enable=True):
        # Make sure inputs to body are valid
        assert ((payload_id == 0) or (payload_id == 1))
        assert (opcode in ['add', 'sub', 'add_even'])
        assert (lock_id < len(self.locks))
        
        if lock_enable:
            self.locks[lock_id].acquire()
        
        # Do something depending on the opcode
        for i in range(1000000):
            if opcode == 'add':
                self.payloads[payload_id] += i
            elif opcode == 'sub':
                self.payloads[payload_id] -= i
            elif opcode == 'add_even':
                if i % 2:
                    self.payloads[payload_id] += i
                    
        if lock_enable:
            self.locks[lock_id].release()
        
        print(f"In body w/ id = {identifier}, and my payload is payload{payload_id} = {self.payloads[payload_id]}")
        return

Without **multi-threading**, thread body works as expected with Single thread test 

In [3]:
lock = threading.Lock()

mutex_demo = MutexExample(0, 0, [lock])
mutex_demo.thread_body(payload_id=0, lock_id = 0, opcode='add', identifier='A')
mutex_demo.thread_body(payload_id=0, lock_id = 0, opcode='sub', identifier='B')
mutex_demo.thread_body(payload_id=0, lock_id = 0, opcode='add_even', identifier='C')

In body w/ id = A, and my payload is payload0 = 499999500000
In body w/ id = B, and my payload is payload0 = 0
In body w/ id = C, and my payload is payload0 = 250000000000


## 1.2. Test
### 1.2.1. No mutex
We are testing a multiple thread program, mutex **disabled**. We hope to get 0 because we add, then subtract. 
The final result should be 0

In [4]:
lock = threading.Lock()
mutex_demo = MutexExample(0, 0, [lock])

thread1 = threading.Thread(target=mutex_demo.thread_body, args=(0, 0, 'add', 'A', False))
thread2 = threading.Thread(target=mutex_demo.thread_body, args=(0, 0, 'sub', 'B', False))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

In body w/ id = A, and my payload is payload0 = 209890264712
In body w/ id = B, and my payload is payload0 = 178872846567


As expected, t1 and t2 are in chaos, we cannot get any meaningful results

### 1.2.2. Mutex enabled
Let's try **enable** mutexes

In [6]:
lock = threading.Lock()
mutex_demo = MutexExample(0, 0, [lock])

thread1 = threading.Thread(target=mutex_demo.thread_body, args=(0, 0, 'add', 'A', True))
thread2 = threading.Thread(target=mutex_demo.thread_body, args=(1, 0, 'sub', 'B', True))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

In body w/ id = A, and my payload is payload0 = 499999500000
In body w/ id = B, and my payload is payload1 = -499999500000


As expected, we are getting determinsitic results now.